<a href="https://colab.research.google.com/github/Oofone/prod-pyth101/blob/master/Notebooks/Basics/Python_Brush_Up_and_OOPs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Writing Good Code in Python

Python is a highly versatile and simple to learn language. It's good for writing small scripts, running Django or Flask based servers in production, and even for server-less AWS Lambda functions for data transformation and APIs.   

## Problem

The only problem is, **python can sometimes be too versatile** and confusing, *much like the ramblings of a very intelligent person*.

If written half-assedly or without some thought and consideration, it can be quite hard for anyone else to understand or work with your code.

## Solution
We'll work with a Toy iPython notebook, and use it to understand how to follow some simple guidelines to write neat and easily readable Python code **WHILE ALSO COVERING** some simple OOPs principles and practices to get the best of both Python and Object Oriented Programming.

---



### What is all of this about?

So what is the point here? 

This notebook serves as a brush-up to just quickly go through a very small python module and understand how to structure your code and make it readable and easy to understand and collaborate with. 

We'll breakdown the following things after this individually:

* Logging
* Data Validation
* Metaclasses and Abstract Classes
* Enforcing Stricter Typing (when necessary)
* Unit-tests

After that, we'll **integrate a Toy Application** with Flask to make a simple Server. 

Finally we'll cover the last two topics: 

1. Dependency Injection (Multiple Encryption implementations)
2. Creating a pipeline with CircleCI for CI/CD to get your code from your laptop to a production host where it can be accessed by people around the world. 

## Cryptography Practice Notebook

Just a simple notebook to practice some crypto functions you'd learn in any basic CS Program or Course.

### Dependency Installation

As always with python, any non-included library module can simply be installed with pip. 

Generally you'd have a dependencies file such as ```requirements``` in your directory and you can simply run ```pip install -r requirements``` to install all your dependencies.

A sample of your ```requirements``` file:

```
beautifulsoup4==4.6.3
certifi==2018.8.24
```

**Since this is an iPython notebook**, we'll simply use a code frame to install and import our global dependencies.

**Note:** Based on your building/deployment framework or pipeline this dependency management system could be similar or very different. 

In [6]:
!pip install request
!pip install bs4
!pip install validation

import logging
import unittest

logging.basicConfig(filename="application.log",
                    filemode='a',
                    format='%(asctime)s,%(msecs)d %(name)s %(levelname)s %(message)s',
                    datefmt='%H:%M:%S',
                    level=logging.DEBUG)
logging.info("Imported dependencies and initialized logger")

### Simple Ceasar Cipher

A Ceasar Cipher is a simple substitution cipher. 
A plain-text String and key are provided. Each character is substituted with a character displaced by the number key.

| Plain-text | Key | Encoded Value |
|------------|-----|---------------|
|A           |1    | B             |
|B           |3    | E             |
|C           |25   | B             |

#### Utilities and Modularity

It's important to structure and modularize your code.

Let's start with a utility function.
We'll have a function that performs the main substitution as a displacement (with the key providing length of displacement) and a roll-over accross the first and last alphabets, meaning: 
* 'a' -> 'z'
* 'z' -> 'a'

We'll define a ```Util``` Class which can house all of our utility code.

We'll write a ```static``` function implemented with the ```@staticmethod``` annotation since this function is not an _instance method_ and is a method which is bound to the class and not the object of the class

A **simple paradigm** to write any logical function is as follows:

1. Any logging of input data if necessary (like a requestId).
1. Input data validation as necessary (checking data-types, constraints)
1. Core module logic; the actual logic of the function.
1. Output data validation as necessary
1. Return output.

In [0]:
from validation import validate_text, validate_int

class Util():
  """Utility class containing some utility functions.
  
  function displace_with_rollover: static
  """

  @staticmethod
  def displace_character_with_rollover(input_character, distance):
    """displace_with_rollover Documentation:

    Takes a character and displaces it by given number modulo 26.
    Rolls-over from "z" to the "a", or "a" to "z".
    Returns the case-corrected Character.
    
    :param input_character: A character to apply key-displacement to.
    :type input_character: str

    :param distance: Distance to shift the given input_character by.
    :type distance: int

    :returns: Output character after transformation
    :rtype: str
    """

    logging.info("Displacing %s with key %s", input_character, distance)
    validate_text(input_character, min_length=1, max_length=1, pattern='[a-zA-Z]', required=True)
    validate_int(distance, required=True)

    output = chr(ord('a') + ((ord(input_character.lower()) - ord('a') + distance) % 26))

    return output.lower() if input_character.islower() else output.upper()

####  Unit-Tests 101

A good coding standard is to write some unit-test for a piece of completed code to ensure that further changes to the code don't break existing logic and expectations.

**In fact** it's best to **_start with the unit-tests first_**, to understand the requirements of your code and build the logic as you execute your tests to ensure you're going in the right direction. 

#### What is a Unit-Test?

In the simplest terms: A unit-test is a piece of code (Software Testing method) that is used to test the logic of individual units of source code. 

Now let's write a few unit-tests to ensure that the code for the above ```Util``` Class doesn't break!

In [27]:
class UtilTest(unittest.TestCase):
  
  def test_displace_with_rollover_happy_case_positive_key(self):
    self.assertEqual(Util.displace_character_with_rollover('a', 2), 'c')
  
  def test_displace_with_rollover_happy_case_negative_key(self):
    self.assertEqual(Util.displace_character_with_rollover('a', -1), 'z')

  def test_displace_with_rollover_validation_character_input_excess_length(self):
    self.assertRaises(ValueError, Util.displace_character_with_rollover, "abcd", 1)

unittest.main(argv=['first-arg-is-ignored'], exit=False)

...
----------------------------------------------------------------------
Ran 3 tests in 0.005s

OK


#### Implementation

Here we'll implement the actual code to encrypt and decrypt the Ceasar Cypher using our displacement utility function.

We'll be working with three main instance variables:

* target: A variable that stores the string to transform
* key: A variable that stores the Key to use to transform the target
* ouput: A variable to store the immediate output of processing

They are implemented here as instances of ```property```. Go [here](https://docs.python.org/3/howto/descriptor.html#properties) to read more about properties and how Decorators work.

Essentially we're using these as private variables. Any variable ```name``` preceded by two underscores like ```__name``` will be regarded by Python as a private variable. Also the function ```__init__(self, ...args...)``` acts as the constructor for the Class. 

In [0]:
from validation import validate_text, validate_int

class CeasarCipher():
  """Ceasar Cipher Implementation Class

  A class used to implement a simple Ceasar Cipher.

  :param target: The target variable stores the String on which encryption and decryption operations are performed on.
  Additionally this string must contain at-least one alphabetic characters and no non-alphabetical characters.
  :type target: str
  
  :param key: The key contains an integer key that is used to perform the substitution.
  :type key: int

  :param output: The output variable stores the output of the operation. 
  :type output: str
  """

  @property
  def target(self):
    return self.__target

  @target.setter
  def target(self, target):
    try:
      validate_text(target, min_length=1, required=True, pattern='[a-zA-Z]+')
    except ValueError as v:
      logging.error("Failure setting target %s", target)
      raise ValueError("The target String must consist of at least one alphabet and non-alphabetical characters are allowed")

    self.__target = target

  @property
  def key(self):
    return self.__key

  @key.setter
  def key(self, key):
    validate_int(key, required=True)
    self.__key = key

  @property
  def output(self):
    return self.__output

  def __init__(self, target=None, key=None):
    if target is not None: self.target = target 
    if key is not None: self.key = key

  def encrypt(self):
    """ encrypt: Function to perform encryption on target variable

    Output is stored in the output variable as well.

    :returns: The encrypted String
    :rtype: str
    """

    logging.info("Encrypting target %s with key %s", self.target, self.key)
    self.__output = ''.join([Util.displace_character_with_rollover(
        input_character=character, 
        distance=self.key
    ) for character in list(self.target)])
    return self.output
  
  def decrypt(self):
    """ decrypt: Function to perform decryption on target variable

    Output is stored in the output variable as well.
    
    NOTE: The substitution is automatically done in the reverse direction. 

    :returns: The decrypted String
    :rtype: str
    """

    logging.info("Decrypting target %s with key %s", self.target, self.key)
    self.__output = ''.join([Util.displace_character_with_rollover(
        input_character=character, 
        distance=-int(self.key)
    ) for character in list(self.target)])
    return self.output

Let's test out our CeasarCipher Class

In [29]:
cipher = CeasarCipher(target="HelloWorld", key=10)
print("Encrypted:", cipher.encrypt())

cipher.target = cipher.output
print("Decrypted:", cipher.decrypt())

Encrypted: RovvyGybvn
Decrypted: HelloWorld


##### An interesting point to note about private variables.

We can't set the output value on any instance of CeasarCipher, as it's a private data member without any setter implement

In [15]:
cipher.output = "MyOutput"

AttributeError: ignored